<a href="https://colab.research.google.com/github/Aakash3101/Deep-Learning/blob/master/ML/Deep_learning/pytorch_tutorials/cifar10/CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Creating the model**

### Observation Table
---
Model 1:

```
```

---
| Model S.no | I (Lr, Epochs) | II (Lr, Epochs) | optimizer | val_acc | test_acc |
| --- | --- | --- | --- | --- | --- |
| 1 | (0.1, 20) | (0.01, 10) | SGD | 55.37 | 54.74 |
| 1 | (0.2, 20) | (0.1, 10) | SGD | 49.40 | 48.86 |
| 1 | (0.5, 20) | (0.1, 10) | SGD | 48.87 | 50.66 |
| 1 | (0.1, 30) | (0.01, 10) | SGD | 55.66 | 55.31 |
| 1 | (0.001, 20) | (0.0001, 10) | Adam | 53.04 | 53.34 |
| 1 | (0.0001, 20) | (1e-5, 10) | Adam | 51.62 | 52.17 |

---

In [ ]:
torch.cuda.empty_cache()